In [88]:
import os
import networkx as nx
import json
from datetime import datetime
import dateutil.parser as parser
import pandas as pd
import sys
import multiprocessing as mp
import numpy as np
import pickle

In [2]:
TRACE_DATA_DIR = '/mnt/raid0_ssd_8tb/rimon/gemini1_datasets/trace/'

In [3]:
all_trace_files = [os.path.join(TRACE_DATA_DIR, f) for f in os.listdir(TRACE_DATA_DIR)]

In [5]:
temp1 = all_trace_files[0]
temp2 = all_trace_files[1]

In [7]:
df1 = pd.read_json(temp1, lines=True)
df2 = pd.read_json(temp2, lines=True)

In [55]:
df1

,datum,CDMVersion,source
0,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
1,{'com.bbn.tc.schema.avro.cdm18.SrcSinkObject':...,18,SOURCE_LINUX_SYSCALL_TRACE
2,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
3,{'com.bbn.tc.schema.avro.cdm18.SrcSinkObject':...,18,SOURCE_LINUX_SYSCALL_TRACE
4,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_SYSCALL_TRACE
...,...,...,...
4999995,{'com.bbn.tc.schema.avro.cdm18.Subject': {'uui...,18,SOURCE_LINUX_BEEP_TRACE
4999996,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_BEEP_TRACE
4999997,{'com.bbn.tc.schema.avro.cdm18.Subject': {'uui...,18,SOURCE_LINUX_BEEP_TRACE
4999998,{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid'...,18,SOURCE_LINUX_BEEP_TRACE


In [58]:
df2['datum'].iloc[0]

{'com.bbn.tc.schema.avro.cdm18.Event': {'uuid': '0E695069-A079-882E-9F89-AB321FDBF1C9',
  'sequence': {'long': 1937130142},
  'type': 'EVENT_MPROTECT',
  'threadId': {'int': 22247},
  'hostId': '3120B2A9-057E-E4EB-4BB9-154983D2C063',
  'subject': {'com.bbn.tc.schema.avro.cdm18.UUID': '204EB826-1787-B9D1-CBD9-2D9015AA598E'},
  'predicateObject': {'com.bbn.tc.schema.avro.cdm18.UUID': '00EAB82E-BA0D-52DE-3394-486F11558ED8'},
  'predicateObjectPath': None,
  'predicateObject2': None,
  'predicateObject2Path': None,
  'timestampNanos': 1523469878202000000,
  'name': None,
  'parameters': None,
  'location': None,
  'size': None,
  'programPoint': None,
  'properties': {'map': {'protection': '5'}}}}

In [15]:
unique_datum1 = set([])
df1.apply(lambda x: unique_datum1.add(next(iter(x['datum']))), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
4999995    None
4999996    None
4999997    None
4999998    None
4999999    None
Length: 5000000, dtype: object

In [17]:
unique_datum2 = set([])
df2.apply(lambda x: unique_datum2.add(next(iter(x['datum']))), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
4999995    None
4999996    None
4999997    None
4999998    None
4999999    None
Length: 5000000, dtype: object

In [20]:
print(unique_datum1 == unique_datum2)
unique_datum1

True


{'com.bbn.tc.schema.avro.cdm18.Event',
 'com.bbn.tc.schema.avro.cdm18.FileObject',
 'com.bbn.tc.schema.avro.cdm18.MemoryObject',
 'com.bbn.tc.schema.avro.cdm18.NetFlowObject',
 'com.bbn.tc.schema.avro.cdm18.SrcSinkObject',
 'com.bbn.tc.schema.avro.cdm18.Subject',
 'com.bbn.tc.schema.avro.cdm18.UnitDependency',
 'com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject'}

In [ ]:
df1['datum'].apply(lambda x: list(x.keys())).explode().value_counts()

com.bbn.tc.schema.avro.cdm18.Event                3344976
com.bbn.tc.schema.avro.cdm18.SrcSinkObject         675012
com.bbn.tc.schema.avro.cdm18.Subject               453701
com.bbn.tc.schema.avro.cdm18.UnitDependency        394325
com.bbn.tc.schema.avro.cdm18.MemoryObject           88522
com.bbn.tc.schema.avro.cdm18.NetFlowObject          28082
com.bbn.tc.schema.avro.cdm18.FileObject             14118
com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject       1264
Name: datum, dtype: int64

In [52]:
df2['datum'].apply(lambda x: list(x.keys())).explode().value_counts()

com.bbn.tc.schema.avro.cdm18.Event                3262915
com.bbn.tc.schema.avro.cdm18.SrcSinkObject         666849
com.bbn.tc.schema.avro.cdm18.Subject               500649
com.bbn.tc.schema.avro.cdm18.UnitDependency        467589
com.bbn.tc.schema.avro.cdm18.MemoryObject           56515
com.bbn.tc.schema.avro.cdm18.NetFlowObject          32894
com.bbn.tc.schema.avro.cdm18.FileObject             11845
com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject        744
Name: datum, dtype: int64

In [54]:
print(3344976 / 5000000)
print(3262915 / 5000000)
# around 66% are event objects - good

0.6689952
0.652583


In [59]:
# do some timestamp analysis 
normalized_df1 = pd.json_normalize(df1['datum'])

In [60]:
normalized_df1

,com.bbn.tc.schema.avro.cdm18.Event.uuid,com.bbn.tc.schema.avro.cdm18.Event.sequence.long,com.bbn.tc.schema.avro.cdm18.Event.type,com.bbn.tc.schema.avro.cdm18.Event.threadId.int,com.bbn.tc.schema.avro.cdm18.Event.hostId,com.bbn.tc.schema.avro.cdm18.Event.subject.com.bbn.tc.schema.avro.cdm18.UUID,com.bbn.tc.schema.avro.cdm18.Event.predicateObject.com.bbn.tc.schema.avro.cdm18.UUID,com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath,com.bbn.tc.schema.avro.cdm18.Event.predicateObject2,com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path,...,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.baseObject.epoch.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.baseObject.properties.map.pid,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sourceFileDescriptor.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sinkFileDescriptor.int,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sourceUUID,com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.sinkUUID,com.bbn.tc.schema.avro.cdm18.Event.properties.map.operation,com.bbn.tc.schema.avro.cdm18.Subject.parentSubject,com.bbn.tc.schema.avro.cdm18.Subject.properties.map.seen time,com.bbn.tc.schema.avro.cdm18.Event.location.long
0,B7CF3A75-822B-CA7B-9FAF-074E5EF1FE90,1.908210e+09,EVENT_SENDMSG,21790.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,6881BEAE-AEDC-B80D-F325-67C69EA9D711,17B28CB0-EAC8-10B3-5BE1-B672D611BD9B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7FEDB19D-70FE-A735-10AC-C1D2AF37312C,1.908210e+09,EVENT_RECVMSG,21792.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,5BA646C2-52E0-FD93-4763-09358A07202A,0E06BFEC-893C-1E56-0122-2B3FF4E00701,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44F8544B-E70A-F803-196F-50963746BE35,1.908210e+09,EVENT_SENDMSG,21792.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,5BA646C2-52E0-FD93-4763-09358A07202A,5DD8F7FB-96F3-E61D-24D7-E4DD70508B2D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999996,9A9D80D7-FCBE-8D1D-3079-00F2755167AC,1.926364e+09,EVENT_UNIT,22208.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,8E0F9235-C338-73C4-6B46-411935CC7D18,39D9CDA5-BFCB-0BD4-58E0-17A0CDB8F62A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999998,26D5D084-06F9-EA33-10C4-25D64728075A,1.926364e+09,EVENT_UNIT,22193.0,3120B2A9-057E-E4EB-4BB9-154983D2C063,53D666B1-70C2-F82F-FE45-E9FF6F00EE6A,091BF32A-DC5D-8DFA-3D9C-9F936FDB205D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
normalized_df1.min(axis=0)

/tmp/ipykernel_1734869/1322253572.py:1: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  normalized_df1.min(axis=0)


com.bbn.tc.schema.avro.cdm18.Event.sequence.long                           1.908210e+09
com.bbn.tc.schema.avro.cdm18.Event.threadId.int                            1.051000e+03
com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath                              NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2                                 NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path                             NaN
com.bbn.tc.schema.avro.cdm18.Event.timestampNanos                          1.523462e+18
com.bbn.tc.schema.avro.cdm18.Event.name                                             NaN
com.bbn.tc.schema.avro.cdm18.Event.parameters                                       NaN
com.bbn.tc.schema.avro.cdm18.Event.location                                         NaN
com.bbn.tc.schema.avro.cdm18.Event.size.long                               0.000000e+00
com.bbn.tc.schema.avro.cdm18.Event.programPoint                                     NaN
com.bbn.tc.schema.avro.cdm18.Src

In [63]:
normalized_df1.max(axis=0)

/tmp/ipykernel_1734869/2872451825.py:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  normalized_df1.max(axis=0)


com.bbn.tc.schema.avro.cdm18.Event.sequence.long                           1.926364e+09
com.bbn.tc.schema.avro.cdm18.Event.threadId.int                            2.991300e+04
com.bbn.tc.schema.avro.cdm18.Event.predicateObjectPath                              NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2                                 NaN
com.bbn.tc.schema.avro.cdm18.Event.predicateObject2Path                             NaN
com.bbn.tc.schema.avro.cdm18.Event.timestampNanos                          1.523465e+18
com.bbn.tc.schema.avro.cdm18.Event.name                                             NaN
com.bbn.tc.schema.avro.cdm18.Event.parameters                                       NaN
com.bbn.tc.schema.avro.cdm18.Event.location                                         NaN
com.bbn.tc.schema.avro.cdm18.Event.size.long                               3.295642e+06
com.bbn.tc.schema.avro.cdm18.Event.programPoint                                     NaN
com.bbn.tc.schema.avro.cdm18.Src

In [90]:
# min timestamp for events from df1: Wed Apr 11 2018 11:53:20 GMT-0400 (Eastern Daylight Time)
# max timestamp for events from df1: Wed Apr 11 2018 12:43:20 GMT-0400 (Eastern Daylight Time)

# max and min timestamps from all files:
timestamps_df = pd.read_csv('data/timestamps.txt', sep='\t')

{}
